### 矢量数据转为栅格数据

In [1]:
from osgeo import gdal, ogr


In [2]:
path_vec = 'data/Section-5/s2_chenggong_mndwi_thre025_polygon.shp'
path_raster = 'data/Section-5/s2_chenggong_mndwi_polygon2raster.tif'


### 主要函数：
#### gdal.RasterizeLayer（dataset, bands, layer, options）
dataset:输出栅格数据；   
bands:指定要更新的波段，为list数据类型;   
layer:指定矢量图层；   
options:控制栅格化的参数。  
参考：https://gdal.org/api/python/osgeo.gdal.html 

In [3]:
# Open the data source and read in the extent
dsource_vec = ogr.Open(path_vec)
layer_vec = dsource_vec.GetLayer()
proj_vec = layer_vec.GetSpatialRef()
proj_vec_wkt = proj_vec.ExportToWkt()
print(proj_vec_wkt)
x_min, x_max, y_min, y_max = layer_vec.GetExtent()


PROJCS["WGS 84 / UTM zone 47N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32647"]]


In [4]:
## Define pixel_size and geo_trans
pixel_size = 20
geo_trans = (x_min, pixel_size, 0, y_max, 0, -pixel_size)
# Create the destination data source
x_size = int((x_max - x_min) / pixel_size)
y_size = int((y_max - y_min) / pixel_size)
dset_raster = gdal.GetDriverByName('GTiff').Create(path_raster, 
                                                    xsize=x_size, 
                                                    ysize=y_size, 
                                                    bands=1, 
                                                    eType=gdal.GDT_Int16
                                                    )
dset_raster.SetGeoTransform(geo_trans)
dset_raster.SetProjection(proj_vec_wkt)


0

In [5]:
# Rasterize
gdal.RasterizeLayer(dataset=dset_raster, bands=[1], layer=layer_vec, options=["ATTRIBUTE=DN"])
dset_raster = None



### 课后练习
#### 快捷方式（控制台命令）：gdal_rasterize/gdal.Rasterize()
参考：https://gdal.org/programs/gdal_rasterize.html  



In [6]:
# path_vec = 'data/Section-5/s2_chenggong_mndwi_polygon.shp'
# path_raster = 'data/Section-5/s2_chenggong_mndwi_polygon2raster.tif'
# !gdal_rasterize -b 1 -l s2_chenggong_mndwi_polygon $path_vec $path_raster -a DN

